# Wrapping a subset of a very large library

In this example, we aim at presenting the methodology to wrap only a subset of a library.
For the purpose of illustration and to demonstrate the usage of **AutoWIG** for very complex libraries, we considered the wrapping of the **Clang** library, a complete *C*/*C++* compiler.
**Clang** is a great tool, but its stable *Python* interface (i.e. **libclang**) is lacking some useful features that are needed by **AutoWIG**.
In particular, class template specializations are not available in the abstract syntax tree.
Fortunately, most of the classes that would be needed during the traversal of the *C++* abstract syntax tree are not template specializations.
We therefore proposed to bootstrap the **Clang** *Python* bindings with a first version of **AutoWIG** that uses only **libclang**.
The new **Clang** *Python* interface, produced by **AutoWIG**, is **PyClangLite**.
**PyClangLite** is able to wrap class template specializations.
As for **libclang**, this interface is proposed only for a subset of the **Clang** library sufficient enough for proposing the new `pyclanglite` `parser`.

First, ensure that the **LLVM**/**Clang** technologies are installed on your computer and if there were built with `RTTI`.

In [1]:
import subprocess
subprocess.check_call(['llvm-config', '--version'])
subprocess.check_call(['clang++', '--version'])
if not subprocess.check_output(["llvm-config", "--has-rtti"]).strip() == 'YES':
    raise Exception('')

Then, clone the **PyClangLite**  repository into the **PyClangLite** directory (we first erase it, if present)

In [3]:
from path import path
from shutil import rmtree
directory = path('PyClangLite')
if directory.exists():
    rmtree(directory.relpath('.'))
from git import Repo
repo = Repo.clone_from('https://github.com/StatisKit/PyClangLite.git', directory.relpath('.'))

This repository already has wrappers, we therefore need to remove them

In [4]:
for wrapper in directory.walkfiles('*.cpp'):
    wrapper.unlink()

It is assumed that both **LLVM** and **Clang** has been installed with the same installation prefix.
We can therefore easily obtain all **Clang** headers that need to be parsed.

In [5]:
includedir = path(subprocess.check_output(["llvm-config", "--prefix"]).strip())/'include'
headers = list((includedir/'clang').walkfiles('*.h'))

In [13]:
import autowig
asg = autowig.AbstractSemanticGraph()
autowig.parser.plugin = 'libclang'

In [16]:
autowig.__path__


['/home/pfernique/Desktop/AutoWIG/src/py/autowig']

In [20]:
import autowig
asg = autowig.AbstractSemanticGraph()
autowig.parser.plugin = 'libclang'
autowig.parser(asg, headers,
               flags=['-x', 'c++', '-std=c++11', '-D__STDC_LIMIT_MACROS', '-D__STDC_CONSTANT_MACROS', '-I' + str(includedir.abspath())],
               bootstrap=False)

ImportError: cannot import name preprocessing

In [ ]:
rmtree(directory.relpath('.'))